# Dispatch for `AbstractSite` objects

Since a lot of information about site objects is already encoded into their type signature, this signature can be used in dispatch to greatly reduce the number of `if` statements as well as greatly improve the performance of your code.

This tutorial aims at providing an overview how to use dispatch for `AbstractSite` objects as well as giving some examples.

In [ ]:
using LatPhysBase

### Reminder: type structure for `AbstractSite`

Site objects are in general subtype of `AbstractSite{L,D}` where the parametric types `L` and `D` give the **label type** as well as the **spatial dimension** of the site location respectively.

E.g. an object that is subtype of `AbstractSite{String,2}` would describe a site with `String` labels in d=2.



### How to use dispatch in function headers

If one wants to dispatch on a certain type signature, the first step is to include the types into the function header. One can do so by the following syntax:
```
function myfunction(s::S) where {S}
    # do something
end
```
In this way, function `myfunction` accepts any argument, since `S` is (implictly) subtype of `Any`.
To specialise to `AbstractSite` objects, one can use
```
function myfunction(s::S) where {S<:AbstractSite}
    # do something
end
```
which would only allow `s` to be of a subtype of `AbstractSite`, i.e. to be a site.
Further specialization can be done by explicitly introducing the parametric types of `AbstractSite`.
```
function myfunction(s::S) where {L,D, S<:AbstractSite{L,D}}
    # do something
end
```
as these parametric types can be adjusted to the individual case as shown in the examples below.




# Examples

Let's discuss some applications in the following


### Example 1 - Plotting (Basics)

Imagine the case in which you wanted to construct a plotting function that plots any given site object. However, depending on the spatial dimension of the site given, you have to call a different function of the plotting library (e.g. `scatter` vs. `scatter3D` in `PyPlot`). In this case, it is helpful to construct a plotting syntax that acceps any site and determines which function to call by dispatch.

We start by implementing a _Fallback_ function that is used if no specialised dispatch signature is identified. Usually a fallback is only used if we encounter a case that is not expected, so we print an according message.

In [ ]:
# Fallback function
function plotSite(s::S) where {S}
    println("Fallback is used - unexpected type for plotting site: ", S)
end

Next, we implement the two functions that are implementing the plotting for d=2 and d=3. These functions only accept sites of that specific dimension.

In [ ]:
# plotting for d=2
function plotSite(s::S) where {L,S<:AbstractSite{L,2}}
    println("plotting in 2d is used. Making a 2d dot at ", point(s))
end

# plotting for d=3
function plotSite(s::S) where {L,S<:AbstractSite{L,3}}
    println("plotting in 3d is used. Making a 3d dot at ", point(s))
end

Last but not least we implement functions that give suitable error messages forsites with dimensions not captured by our two special functions d=2 and d=3. Note that explicitly requiring a site type here is crucial to override the Fallback function above.

In [ ]:
# plotting for any d --> dont know how it works
function plotSite(s::S) where {L,D,S<:AbstractSite{L,D}}
    println("don't know how plotting works in d=",D)
end

Finally, let's test our example with some specific site objects.

In [ ]:
# test dispatch on dimension

# d=1 --> plotting unknown
site_1 = newSite(Site{String,1}, [0.0,], "mysite")
plotSite(site_1)

# d=2
site_2 = newSite(Site{String,2}, [0.0, 0.0], "mysite")
plotSite(site_2)

# d=3
site_3 = newSite(Site{String,3}, [0.0,0.0,0.0], "mysite")
plotSite(site_3)

# d=4 --> plotting unknown
site_4 = newSite(Site{String,4}, [0.0,0.0,0.0,0.0], "mysite")
plotSite(site_4)



# test dispatch on a non-site type --> Fallback
non_site = [1.0,0.0]
plotSite(non_site)

One of the advantages of the dispatch solution here, is that we can use the same syntax on all sites with an efficient implementation. This can be further demonstrated in the example below, where we put all sites into a single Array.

Note that in the example below, the interpreter has to use dynamic dispatch, therefore the code does not run efficient performance wise. However on non-performce intense parts of the code, this is fine.

In [ ]:
# create a list of sites
site_list = [site_1, site_2, site_3, site_4, non_site]

# plot all sites of the list
for s in site_list
    plotSite(s)
end

### Example 2 - Plotting (Advanced)

We can extend the above example of dispatch for plotting by incorparating dispatch on site label within the plotting. Imagine, you wanted to plot sites differently for different labels, e.g. give a text next to the site if the label is a `String` or `Symbol` or chose the site markersize if it is a real number. This can be implemented as follows.

For simplicity, consider only the d=2 plotting function. From the above example we already have everything set up, so now we want to distinguish by the type of the site labels. We again implement a structure like fallback and specific functions. If you want the same functionality, you could simply implement these functions for d=3 alike.

In [ ]:
# Fallback plotting for d=2 (overrides function above)
function plotSite(s::S) where {L,S<:AbstractSite{L,2}}
    println("plotting in 2d is used. Making a 2d dot at ", point(s), ", label has unsupported type: ", L)
end


# plotting for d=2 with String or Symbol label
function plotSite(s::S) where {L<:Union{String,Symbol},S<:AbstractSite{L,2}}
    println("plotting in 2d is used. Making a 2d dot at ", point(s), ", label gets text \"", label(s), "\"")
end

# plotting for d=2 with Real number as label
function plotSite(s::S) where {L<:Real,S<:AbstractSite{L,2}}
    println("plotting in 2d is used. Making a 2d dot at ", point(s), ", setting markersize=", label(s))
end

In [ ]:
plotSite(  newSite(Site{String,2}, [0.0, 0.0], "mysite")  )
plotSite(  newSite(Site{Symbol,2}, [0.0, 0.0], :mysite)  )
plotSite(  newSite(Site{Float64,2}, [0.0, 0.0], 2.0)  )
plotSite(  newSite(Site{Int64,2}, [0.0, 0.0], 3)  )